In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name='Assistência social', nrows=29, usecols=['Cod Uf', 'EASS01','EASS03','EASS04', 'EASS05', 'EASS06'])
df

,Cod Uf,EASS01,EASS03,EASS04,EASS05,EASS06
0,11,Secretaria em conjunto com outras políticas se...,Masculino,50,Branca,Ensino superior completo
1,12,Secretaria exclusiva,Masculino,43,Branca,Ensino superior completo
2,13,Secretaria exclusiva,Feminino,45,Pardo,Ensino superior completo
3,14,Secretaria em conjunto com outras políticas se...,Feminino,37,Branca,Especialização
4,15,Secretaria em conjunto com outras políticas se...,Masculino,58,Branca,Especialização
5,16,Secretaria em conjunto com outras políticas se...,Feminino,54,Branca,Especialização
6,17,Secretaria em conjunto com outras políticas se...,Feminino,54,Branca,Ensino superior completo
7,21,Secretaria em conjunto com outras políticas se...,Feminino,61,Branca,Especialização
8,22,Secretaria em conjunto com outras políticas se...,Feminino,49,Branca,Ensino superior completo
9,23,Secretaria em conjunto com outras políticas se...,Masculino,52,Branca,Especialização


In [4]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2020.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])

Renomeando as colunas

In [5]:
df= df.rename(columns={'Cod Uf':'cod_uf',
                        'EASS01':'caracterizacao_orgao_gestor',
                        'EASS03':'genero',
                        'EASS04' :'idade',
                        'EASS05':'cor_raca',
                        'EASS06':'grau_instrucao'}) 

In [6]:
##adicionando a coluna de 'ano' 
df['ano']=2018

In [7]:
x= uf.pivot_table(columns=('Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação'), aggfunc='size')


In [8]:
uf = pd.DataFrame(x).reset_index()[['Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação']]

In [9]:
df = df.merge(uf, right_on='Código da Unidade da Federação',left_on='cod_uf') #juntando os dataframes, adicionando sigla e nome das UFs


In [10]:
df = df.drop(['Código da Unidade da Federação'], axis=1) #eliminando coluna repetida

In [11]:
df = df.rename(columns={'Sigla da Unidade da Federação':'sigla_uf',
                        'Nome da Unidade da Federação':'uf'}) #padronizando as colunas

In [12]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [13]:
#padronizando os dados da coluna 'cor_raca'
df['cor_raca'] = np.where(df['cor_raca']=='Pardo','Parda',df['cor_raca'])

In [14]:
limites = [0, 30, 50,65,100] #criando uma nova coluna (faixa_etaria) com base na coluna 'idade'
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [15]:
# criando dicionário
dict_esco = {'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado'}

In [16]:
df = df.replace({'grau_instrucao':dict_esco})

In [17]:
df= df[['ano', 'sigla_uf','cod_uf', 'uf', 'caracterizacao_orgao_gestor','genero', 'faixa_etaria', 'cor_raca','grau_instrucao']]

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          27 non-null     int64   
 1   sigla_uf                     27 non-null     object  
 2   cod_uf                       27 non-null     int64   
 3   uf                           27 non-null     object  
 4   caracterizacao_orgao_gestor  27 non-null     object  
 5   genero                       27 non-null     object  
 6   faixa_etaria                 27 non-null     category
 7   cor_raca                     27 non-null     object  
 8   grau_instrucao               27 non-null     object  
dtypes: category(1), int64(2), object(6)
memory usage: 2.1+ KB


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'), 
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'), 
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')
        ]


In [ ]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_assistencia_social_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=2a78818d-a88d-4956-8894-6ef5cff4cb7c>